In [25]:
import os, sys
import warnings
import timeit
import numpy as np
import matplotlib.pyplot as plt
import re
import pandas as pd

from collections import defaultdict

In [74]:
data_a=pd.read_excel('../postprocessed-data/dbem3_a_pergene_insertions.xlsx',engine='openpyxl')
data_a=data_a.drop(['Unnamed: 0'],axis=1)

data_b=pd.read_excel('../postprocessed-data/dbem3_b_pergene_insertions.xlsx',engine='openpyxl')
data_b=data_b.drop(['Unnamed: 0'],axis=1)
# data.fillna(1,inplace=True)
data_merged=pd.read_excel('../postprocessed-data/dbem3_merged_pergene_insertions.xlsx',engine='openpyxl')
data_merged=data_merged.drop(['Unnamed: 0'],axis=1)

In [75]:
data_a.head(2)

,Gene name,Chromosome,Start location,End location,Insertion locations,Reads per insertion location,Reads,Insertions
0,YAL069W,I,335,649,"[337, 337, 341, 344, 350, 352, 368, 370, 387, ...","[2, 93, 12, 57, 25, 1, 4, 2, 120, 16, 1, 12, 1...",884,40
1,YAL068W-A,I,538,792,"[567, 571, 572, 578, 579, 589, 595, 687, 691, ...","[71, 5, 1, 1, 140, 8, 31, 137, 214, 19, 3, 1, ...",428,18


In [22]:
data_b.head(2)

,Gene name,Chromosome,Start location,End location,Insertion locations,Reads per insertion location,Reads,Insertions
0,YAL069W,I,335,649,"[336, 340, 349, 372, 377, 382, 386, 386, 387, ...","[3, 327, 87, 1, 3, 12, 39, 1, 43, 6, 3, 45, 1,...",2353,55
1,YAL068W-A,I,538,792,"[559, 561, 564, 565, 569, 570, 570, 571, 573, ...","[6, 1, 2, 2, 147, 9, 154, 17, 177, 1, 132, 5, ...",694,25


In [23]:
data_merged.head(2)

,Gene name,Chromosome,Start location,End location,Insertion locations,Reads per insertion location,Reads,Insertions
0,YAL069W,I,335,649,"[336, 337, 340, 344, 349, 352, 368, 371, 377, ...","[3, 2, 430, 57, 114, 1, 4, 3, 3, 12, 55, 1, 14...",3428,67
1,YAL068W-A,I,538,792,"[559, 561, 564, 568, 570, 570, 571, 573, 576, ...","[6, 3, 2, 202, 9, 154, 22, 161, 2, 1, 296, 12,...",1207,31


In [176]:
import from_excel_to_list

In [131]:
a=from_excel_to_list(data_a['Insertion locations'][0])
b=from_excel_to_list(data_b['Insertion locations'][0])
a_reads=from_excel_to_list(data_a['Reads per insertion location'][0])
b_reads=from_excel_to_list(data_b['Reads per insertion location'][0])

In [132]:
set_a=set(a)
set_b=set(b)
s = set_a.union(set_b)  # union of both sets

# Difference in two sets
diff_element_a = set_b - set_a
diff_element_b = set_a - set_b
  
# union of difference + first list
output = a + list(diff_element_a) + list(diff_element_b)



In [133]:
d = defaultdict(int)
for k in output:
    d[k] += 1
    

In [134]:
merged_insertions=pd.DataFrame(d.items(), columns=['Insertion locations', 'Count'])
merged_insertions.sort_values(by='Count', inplace=True,ascending=False)

In [109]:
a_reads_dict=defaultdict(int)
j=0
for k in a:
    reads=a_reads[j]
    a_reads_dict[k] = reads
    j=j+1



In [142]:
def get_reads_with_insertions(replicate_reads_per_gene,replicate_insertions_per_gene):
    """Get the reads per insertion location for a given replicate

    Parameters
    ----------
    replicate_reads_per_gene : list
        output from from_excel_to_list function
    """
    reads_dict=defaultdict(int)
    j=0
    for k in replicate_insertions_per_gene:
        reads=replicate_reads_per_gene[j]
        reads_dict[k] = reads
        j=j+1
    return reads_dict

In [143]:
a_reads_dict=get_reads_with_insertions(a_reads,a)
b_reads_dict=get_reads_with_insertions(b_reads,b)

In [144]:
a_reads_df=pd.DataFrame(a_reads_dict.items(), columns=['Insertion locations', 'Reads per insertion location'])
b_reads_df=pd.DataFrame(b_reads_dict.items(), columns=['Insertion locations', 'Reads per insertion location'])

In [164]:
a_reads_merged=pd.merge(pd.merge(merged_insertions,a_reads_df,on='Insertion locations'),b_reads_df,on='Insertion locations',how='left')
a_reads_merged.fillna(0,inplace=True)

In [175]:
print(np.sum(b_reads_df['Reads per insertion location'])-np.max(b_reads_df["Reads per insertion location"]))
print(np.sum(a_reads_df['Reads per insertion location'])-np.max(a_reads_df["Reads per insertion location"]))

2249.0
882.0


In [168]:
print(np.sum(a_reads_merged['Reads per insertion location_x']))
print(np.sum(a_reads_merged['Reads per insertion location_y']))

1096.0
239.0


In [156]:
group=a_reads_merged.groupby("Count").sum()
group

,Insertion locations,Reads per insertion location_x,Reads per insertion location_y
Count,,,
1,6110.0,192.0,239.0
2,11623.0,811.0,0.0
3,337.0,93.0,0.0


In [158]:
total=[]
for i in group.index:
    print(i)
    total.append(group.loc[i]['Reads per insertion location_x']*i+group.loc[i]['Reads per insertion location_y']*i)

1
2
3


In [160]:
np.sum(total)

2332.0